In [1]:
#start
import gspread #type:ignore NOQA
import imaplib
import email
import json
from decimal import Decimal

#connecting to GSeets and selecting the file
sa = gspread.service_account(filename="sa_creds.json")

#read credentials from json
with open("ym_creds.json", "r") as f:
    jmCreds = json.load(f)

groceryBrandDoneEmails = [] #list to write to GSheets
def groceryBrandEmails(GroceryBrand: str, emailList):
    for i in emailList:
        f = [GroceryBrand]
        f.append(i)
        groceryBrandDoneEmails.append(f)

sh = sa.open("Grocery")
#selecting sheets:
wksEmails = sh.worksheet("Emails")
wksSummaryMaxima = sh.worksheet("MaximaSummarized")

#getting data from wksEmails sheet
# wksEmails.get()
maximaAlreadyIn = [i[1] for i in wksEmails.get() if 'Maxima' in i[0]]

In [2]:
# Connect to the Yahoo Mail IMAP server
imap_server = imaplib.IMAP4_SSL("imap.mail.yahoo.com")
# Login to the account
imap_server.login(jmCreds["email"], jmCreds["password"])
# Select the "Inbox" folder
imap_server.select("Inbox")

# Search for all emails: status, email_ids = imap_server.search(None, "ALL")
# Search for all emails from MAXIMA
status, email_ids = imap_server.search(None, "FROM noreply.code.provider@maxima.lt")
maximaEmails = email_ids[0].decode().split()

maximaToAdd = [i for i in maximaEmails if i not in maximaAlreadyIn]

In [3]:
maximaToAdd

['5539', '5555']

In [4]:
receiptSummaryDataFULL = []
receiptSummaryData = []
def readMaximaReceiptSummary(EmailId: str, receipt):
    receiptSummaryData.clear()
    for i in range(0, 11):
        receiptSummaryData.append('')
    receiptSummaryData[0] = EmailId
    for i in receipt:
        #Receipt_id
        if '<pre>Kvitas ' in i:
            receiptSummaryData[1] = i.split()[1]
        #BankReceipt_id
        if 'KVITO NR' in i:
            receiptSummaryData[2] = i.split()[2]
        #Receipt_Document#
        if 'DOKUMENTO NR' in i:
            receiptSummaryData[3] = i.split()[2]
        #RRN
        if 'RRN:' in i:
            receiptSummaryData[4] = i.split()[1]
        #Date
        if 'LTF NM ' in i:
            receiptSummaryData[5] = f'{i.split()[3]}-{i.split()[4]}-{i.split()[5]}'
        #Time
        if 'LTF NM ' in i:
            receiptSummaryData[6] = i.split()[6]
        #GroceryBrand
        if 'MAXIMA LT' in i:
            receiptSummaryData[7] = 'MAXIMA LT'
        #Address
        if 'MAXIMA LT, UAB' in i:
            receiptSummaryData[8] = i.split('<br />')[1].split(' Kasa Nr.')[0][:-1]
        #TotalAmount
        if 'Apsipirkimo suma:' in i:
            receiptSummaryData[9] = i.split(': ')[1][:-4]
        #TotalDiscount
        if 'Kvito nuolaidų suma:' in i:
            receiptSummaryData[10] = i.split(': ')[1][:-4]
    receiptSummaryDataFULL.append(list(receiptSummaryData))

processedEmails = []
processedEmail = []
def emailProcessLog(GroceryBrand: str, Email_ids):
    for i in Email_ids:
        processedEmail = [GroceryBrand, i]
        processedEmails.append(list(processedEmail))



In [5]:
#writing email summary
for i in maximaToAdd:
    status, email_data = imap_server.fetch(i, "(RFC822)")
    email_message = email.message_from_bytes(email_data[0][1])
    
    msg1 = email_message.get_payload()[0]
    msg1body = msg1.get_payload(decode=True)
    singleEmail = msg1body.decode().split('\r\n')
    readMaximaReceiptSummary(i, singleEmail)
wksSummaryMaxima.append_rows(receiptSummaryDataFULL)

#writing processed emails to Emails sheet
emailProcessLog('Maxima',maximaToAdd)
wksEmails.append_rows(processedEmails)


{'spreadsheetId': '10-mjgW6dVIMLh5iVe6NbyIbI_N3XZ0mOFGlxilp4W0s',
 'tableRange': 'Emails!A1:B209',
 'updates': {'spreadsheetId': '10-mjgW6dVIMLh5iVe6NbyIbI_N3XZ0mOFGlxilp4W0s',
  'updatedRange': 'Emails!A210:B211',
  'updatedRows': 2,
  'updatedColumns': 2,
  'updatedCells': 4}}

In [13]:
# Disconnect from the server
imap_server.close()
imap_server.logout()

('BYE', [b'IMAP4rev1 Server logging out'])

In [5]:
for i in maximaToAdd:
    status, email_data = imap_server.fetch(i, "(RFC822)")
    email_message = email.message_from_bytes(email_data[0][1])

In [8]:
#get message bu ID
status, email_data = imap_server.fetch("5539", "(RFC822)")
email_message = email.message_from_bytes(email_data[0][1])

In [9]:
msg1 = email_message.get_payload()[0]
msg1body = msg1.get_payload(decode=True)
msg2 = email_message.get_payload()[0]
msg2body = msg2.get_payload(decode=True)
msg3 = email_message.get_payload()[0]
msg3body = msg3.get_payload(decode=True)

In [10]:
# print(msg1.get_payload(decode=True), msg2.get_payload(decode=True))
print(msg2body.decode())

<html><body>


<html>

<body>

    <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
            <p>Apsipirkimo suma: 23.01 EUR</p>
            <br />
                <p>Kvito nuolaidų suma: 2.20 EUR</p>
                <br />
                <p> MAXIMOS PINIGŲ likutis: 0.22 EUR </p>
            <p>
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
                &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;

In [7]:
x = msg2body.decode().split('\r\n')
for e, i in enumerate(x):
    print(e, x[e])
# x[10]

0 <html><body>
1 
2 
3 <html>
4 
5 <body>
6 
7     <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
8             <p>Apsipirkimo suma: 22.02 EUR</p>
9             <br />
10                 <p>Kvito nuolaidų suma: 5.85 EUR</p>
11                 <br />
12                 <p> MAXIMOS PINIGŲ likutis: 0.22 EUR </p>
13             <p>
14                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
15                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
16                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
17                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwn

In [11]:
w = msg2body.decode().split('\r\n')
for e, i in enumerate(w):
    print(e, w[e])
# w[10]

0 <html><body>
1 
2 
3 <html>
4 
5 <body>
6 
7     <div class="previewText" style="display:none; max-height: 0px; overflow: hidden">
8             <p>Apsipirkimo suma: 23.01 EUR</p>
9             <br />
10                 <p>Kvito nuolaidų suma: 2.20 EUR</p>
11                 <br />
12                 <p> MAXIMOS PINIGŲ likutis: 0.22 EUR </p>
13             <p>
14                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
15                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
16                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;
17                 &nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwnj;&nbsp;&zwn

In [12]:
for i in w:
    if 'DOKUMENTO NR' in i:
        print(i)

DOKUMENTO NR       478089-230114140925                 #


In [13]:
q = msg2body.decode().split('<pre>Kvitas')
q = q[1].split('======================================================')
q = q[0].replace(' N\r\n', ' N A\r\n')
q = q.replace(' A\r\n','|')
q = q.replace('\r\n','|',1)
q = q.replace('&#160;',' ')
q = ' '.join(q.split())
q = q.split('|')
q = q[1:len(q)-1]

In [14]:
q

['Pienas, 2 % riebumo, be laktozės 1,19 X 5 vnt. 5,95',
 'GUBERNIJOS tamsi ruginė duonos gira 0,85 X 3 vnt. 2,55',
 'Skardinė (depozitinė) 0,10 EUR 0,10 X 3 vnt. 0,30 N',
 'VOLFO ENGELMAN nealkoholinis alaus kokteilis KRIEK, 0% 1,09',
 'Nuolaida:gimtadienis! Nealk.vynui,sidru,alui -30% -0,33',
 'Skardinė (depozitinė) 0,10 EUR 0,10 N',
 'Nealkoholinis UTENOS alaus kokteilis RADLER (mangų skon 1,25',
 'Nuolaida:Nealk.UTENOS alaus kokteilis RADLER -0,30',
 'Skardinė (depozitinė) 0,10 EUR 0,10 N',
 'Gazuotas gėrimas COCA COLA 1,09',
 'PET (depozitinis) 0,10 EUR 0,10 N',
 'Juoda raikyta duona JORĖ 1,39',
 'Nuolaida prekei -0,28',
 'Glaistyti zefyrai 6,49 X 0,374 kg 2,43',
 'Vaisiniai kramtomieji saldainiai TUTTI FRUTTI 0,35',
 'Nuolaida:saldainiams ir šokoladui -30% -0,11',
 'Glaistytas sūrelis MAGIJA su vanile, 24 % rieb. 0,59 X 4 vnt. 2,36',
 'Glaistytas VILKYŠKIŲ vanilės sk. varškės sūrelis, 24% rie 0,59 X 3 vnt. 1,77',
 'Gl. varškės sūrelis NYKŠTUKAS su aguonomis, 23,1% rieb. 0,59',
 'N

In [77]:
for e, i in enumerate(q[0:15]):
    # print(i.split())
    for ee, ii in enumerate(i.split()):
        if ii == 'X' and 'vnt.' in i.split():
        # if ii == 'X' and 'kg' in i.split():
        # if ii == 'X':
            print(ee)
            # print(i[6])
            print(i.split()[ee-1], ii, i.split()[ee+1])
    # if i.split()[len(i.split())-1] != 'N':
        # print(float(i.split()[len(i.split())-1].replace(',','.')))

7
1,19 X 5
6
0,85 X 3
5
0,10 X 3


In [65]:
# for i in q:
#     if 'X' in i.split():
#         print(i)

Pienas, 2 % riebumo, be laktozės 1,19 X 5 vnt. 5,95
GUBERNIJOS tamsi ruginė duonos gira 0,85 X 3 vnt. 2,55
Skardinė (depozitinė) 0,10 EUR 0,10 X 3 vnt. 0,30 N
Glaistyti zefyrai 6,49 X 0,374 kg 2,43
Glaistytas sūrelis MAGIJA su vanile, 24 % rieb. 0,59 X 4 vnt. 2,36
Glaistytas VILKYŠKIŲ vanilės sk. varškės sūrelis, 24% rie 0,59 X 3 vnt. 1,77


In [45]:
from decimal import Decimal
price = "8,33"
# price = Decimal(price.replace(',','.'))
# price + 10.12
price = float(price.replace(',','.'))
str(price + 10.22)


'18.55'

In [87]:
item = ['2023-01-01','20:20:20','5555','231/1119']
#getting final price:
for e, i in enumerate(q):
    finalPrice = i.split()[len(i.split())-1]
    if finalPrice == 'N':
        finalPrice = i.split()[len(i.split())-2]
    finalPrice = float(finalPrice.replace(',','.'))
    print(finalPrice)

5.95
2.55
0.3
1.09
-0.33
0.1
1.25
-0.3
0.1
1.09
0.1
1.39
-0.28
2.43
0.35
-0.11
2.36
1.77
0.59
-0.1
3.79
-0.8


In [90]:
#getting list of items:
items = []
item = ['2023-01-01','20:20:20','5555','231/1119']
for e, i in enumerate(q):
    item = ['2023-01-01','20:20:20','5555','231/1119']
    item.append(i)
    item.append(i.split()[len(i.split())-1])
    items.append(list(item))
items

[['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'Pienas, 2 % riebumo, be laktozės 1,19 X 5 vnt. 5,95',
  '5,95'],
 ['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'GUBERNIJOS tamsi ruginė duonos gira 0,85 X 3 vnt. 2,55',
  '2,55'],
 ['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'Skardinė (depozitinė) 0,10 EUR 0,10 X 3 vnt. 0,30 N',
  'N'],
 ['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'VOLFO ENGELMAN nealkoholinis alaus kokteilis KRIEK, 0% 1,09',
  '1,09'],
 ['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'Nuolaida:gimtadienis! Nealk.vynui,sidru,alui -30% -0,33',
  '-0,33'],
 ['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'Skardinė (depozitinė) 0,10 EUR 0,10 N',
  'N'],
 ['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'Nealkoholinis UTENOS alaus kokteilis RADLER (mangų skon 1,25',
  '1,25'],
 ['2023-01-01',
  '20:20:20',
  '5555',
  '231/1119',
  'Nuolaida:Nealk.UTENOS alaus kokteilis RADLER -0,30',
  '-0,30'],
 ['2023-01-01',
  '